#Identify Pendulum

In [1]:
import sys
sys.path.append('..')

In [2]:
%matplotlib inline  
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
import time

In [3]:
from ctrl.system.tf import PID
from ctrl.block.linear import Gain, Feedback
from ctrl.block.logger import Logger

In [4]:
from ctrl.client import Controller
HOST, PORT = "192.168.10.102", 9999
HOST, PORT = "localhost", 9999
controller = Controller(host = HOST, port = PORT)

In [28]:
controller.reset()
controller.add_sink('logger', 
                    Logger(), 
                    ['clock', 'encoder1','motor1'])

In [29]:
print(controller.info('all'))

> Controller with 5 signal(s), 1 source(s), 1 sink(s), and 2 filter(s)
> period = 0.01s
> signals
  1. clock
  2. motor1
  3. encoder1
  4. pot1
  5. input1
> sources
  1. clock[enabled] >> clock
> filters
  1. motor1 >> dz1[enabled] >> input1
  2. input1 >> model1[enabled] >> encoder1
> sinks
  1. clock, encoder1, motor1 >> logger[enabled]



# Identifying the Friction

In [ ]:
from ctrl.bbb.mpu6050 import Inclinometer 

In [ ]:
controller.set_sink('logger','reset')
with controller:
    controller.set_signal('motor1',70)
    time.sleep(.3)
    #controller.set_signal('motor1',-70)
    #time.sleep(.05)
    controller.set_signal('motor1',0)
    time.sleep(5)
log = controller.read_sink('logger')

In [ ]:
t = log[:,0]
position = log[:,1]
reference = log[:,2]
control = log[:,3]
velocity = np.zeros(t.shape, float)
velocity[1:] = (position[1:]-position[:-1])/(t[1:]-t[:-1])

ind = np.where(t > 0.5)

plt.subplot(2,1,1)
plt.plot(t, position*180/math.pi, '.')
plt.xlabel('t (s)')
plt.ylabel('position (Hz)')
plt.grid()
plt.xlim(.94,4.3)
plt.ylim(-5,5)

plt.subplot(2,1,2)
plt.plot(t, control, '.', t, reference, '-')
plt.xlabel('t (s)')
plt.ylabel('control (%)')
plt.grid()
plt.xlim(.75,6)

In [ ]:
J2 = 0.0000348
wn = 31.25

Td = (4.3-.94)/4
wd = 2*math.pi/Td
zeta = math.sqrt(1-(wd/wn)**2)
print('wd = {:5.3f}, wn = {:5.3f}, zeta = {:5.3f}'.format(wd,wn,zeta))
bTheta = 4*J2*zeta*wn
print('bTheta = {:5.3f}'.format(bTheta))